# 1a

In [29]:
from platform import python_version
print(python_version())

3.7.6


In [30]:
#Libraries
from PIL import Image
import glob
import numpy as np
from numpy import interp, ndarray
from timeit import default_timer as timer
import math

### Load images

In [31]:
image_list = []
image_names = []
for filename in glob.glob('images/*.jpg'): #assuming jpg
    im=Image.open(filename)
    image_names.append(im.filename)
    image_list.append(np.array(im))
    #im.show()
#print(len(image_list))

### Resize images

In [32]:
#resize a list of images by a factor of 5
def resize(images):
    resized = []
    for im in images:
        start = timer()
        #print(im.shape)
        result = np.zeros((int(im.shape[0]/5), int(im.shape[1]/5), im.shape[2]))
        #print(result.shape)
        for dim in range(im.shape[-1]):
            for row in range(int(im.shape[0]/5)):
                for col in range(int(im.shape[1]/5)):
                    square = []
                    for i in range(5):
                        for j in range(5):
                            square.append(im[5*row+i, 5*col+j, dim])
                    #result[row, col, dim] = sum(square)/25 #area average
                    square.sort()
                    result[row, col, dim] = square[12] #median
        resized.append(result)
        end = timer()
        print(end - start)
    return resized

Note: Choosing the median instead of the average reduced the run time by about 28% (from 1.33s to 0.96s per image).

In [33]:
resized_images = resize(image_list)
for im in resized_images:
    Image.fromarray(np.uint8(im)).show()

0.9118854999978794
0.8998844000016106


### Sharpening downscaled images

Since scaling down the image might result in a somewhat less sharp image (especially when using area average or the mean as I did in the implementation above), I tried to apply a sharpening filter on the small image to counter this effect.

In [34]:
#function to clip outliers to rgb range
cut = lambda t : max(min(t, 256), 0)
fnc = np.vectorize(cut)

In [35]:
def kernel(images):
    #sharpen = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    #sharpen = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = np.array([[0,-0.125,0], [-0.125,1.25,-0.125], [0,-0.125,0]])
    print(sharpen)
    results = []
    for im in images:
        temp = np.copy(im)
        for dim in range(im.shape[-1]):
            for row in range(int(im.shape[0])-2):
                for col in range(int(im.shape[1])-2):
                    accumulator = 0.0
                    for i in range(3):
                        for j in range(3):
                            #print(im[row+i, col+j, dim], sharpen[i,j])
                            accumulator += im[row+i, col+j, dim]*sharpen[i,j]
                    temp[row+1, col+1, dim] = accumulator
                    #print(row, col, dim, accumulator)
        results.append(fnc(temp))                   
    return results

Note: Only the last sharpening kernel yielded usable results. The other images had some prominent artifacts. However, this kernel darkens the image noticeably.

In [36]:
final_images = kernel(resized_images)
for im in final_images:
    Image.fromarray(np.uint8(im)).show()

[[ 0.    -0.125  0.   ]
 [-0.125  1.25  -0.125]
 [ 0.    -0.125  0.   ]]


### Testing

Since the first two sharpening kernels left some artifacts on the image (i.e. some value were well beyond the rgb range), the following loop tries to readjust the range (i.e. map the range of image values back to rgb values). Unfortunately, this did not significantly improve the result.

In [9]:
#loop to map range of image values to rgb scale
for im in final_images:
    liste = ndarray.tolist(im)
    liste = [interp(x,[np.amin(im),np.max(im)],[0,256]) for x in liste]
    final = np.array(liste)
    Image.fromarray(np.uint8(final)).show()

### Save images

In [37]:
for l in range(len(image_list)):
    resized_images[l] = Image.fromarray(np.uint8(resized_images[l]))
    resized_images[l].filename = image_names[l]
any(im.save(str(im.filename).replace('600x900', '120x180')) for im in resized_images)

False

# 1b

The following function uses stride = 1 and padding = 0. It assumes a 2D kernel and it  will leave a border of $\lfloor \frac{size(kernel)}{2} \rfloor$ containing unchanged pixels (which should be fine for small kernel, e.g. for size 3x3 only a single pixel border remains). Alternatively, one could also remove these pixels from the image, making the transformed image slightly smaller. 

Note: As you can see the following function is very similar to the sharpening kernel from 1a. The major differences are that this function takes a kernel as input parameter, works with different sized kernels and only applies the kernel to a single image (not a list of images).

In [38]:
def convolute(image, kernel):
    start = timer()
    new_image = np.copy(im)
    offset = int(math.ceil(len(kernel)/2))
    for dim in range(image.shape[-1]):
        for row in range(int(image.shape[0])-2*offset):
             for col in range(int(image.shape[1])-2*offset):
                accumulator = 0.0
                for i in range(len(kernel)):
                    for j in range(len(kernel)):
                        #print(im[row+i, col+j, dim], sharpen[i,j])
                        accumulator += image[row+i, col+j, dim]*kernel[i,j]
                new_image[row+offset, col+offset, dim] = accumulator
                #print(row, col, dim, accumulator)
    end = timer()
    print(end - start)
    return new_image

Blurring

In [39]:
#box_blur = np.array([[1/9,1/9,1/9], [1/9,1/9,1/9], [1/9,1/9,1/9]])
#gaussian_blur = np.array([[1/16,1/8,1/16], [1/8,1/4,1/8], [1/16,1/8,1/16]])
gaussian_blur5 = np.array([[1/256,4/256,6/256,4/256,1/256], [4/256,16/256,24/256,16/256,4/256], 
                            [6/256,24/256,36/256,24/256,6/256], [4/256,16/256,24/256,16/256,4/256],
                            [1/256,4/256,6/256,4/256,1/256]])
blurred_images = []
for im in image_list:
    blurred_images.append(convolute(im, gaussian_blur5)) #aprox runtime per image: 150s

138.0628902999997
137.71480560000055


In [40]:
for im in blurred_images:
    Image.fromarray(np.uint8(im)).show()

Edge detection

In [41]:
#edge_detection = np.array([[1,0,-1], [0,0,0], [-1,0,1]])
edge_detection = np.array([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]])
contour_images = []
for im in image_list:
    contour_images.append(convolute(im, edge_detection)) 

57.91453080000065
59.00540019999971


In [42]:
for im in contour_images:
    Image.fromarray(np.uint8(im)).show()

Sobel kernels

In [43]:
kx = np.array([[-0.25,0,1], [-0.5,0,0.5], [-0.25,0,0.25]])
ky = np.array([[-0.25,-0.5,-0.25], [0,0,0], [0.25,0.5,0.25]])
sobel_images = []
for im in image_list:
    sobel_images.append(convolute(convolute(im, kx), ky)) 

50.93771959999867
50.31189169999925
52.228896800002985
51.57797760000176


In [44]:
for im in sobel_images:
    Image.fromarray(np.uint8(im)).show()

Note: Applying the sobel kernels did actually return an image with more distinct contour lines. However, I think the way I implemented them is wrong, because I apply the two kernels one after the other while the intended way is to apply both kernels on the original image and then somehow combine the two images together. 

Saving images

In [45]:
for l in range(len(image_list)):
    blurred_images[l] = Image.fromarray(np.uint8(blurred_images[l]))
    blurred_images[l].filename = image_names[l]
    contour_images[l] = Image.fromarray(np.uint8(contour_images[l]))
    contour_images[l].filename = image_names[l]
    sobel_images[l] = Image.fromarray(np.uint8(sobel_images[l]))
    sobel_images[l].filename = image_names[l]
any(im.save(str(im.filename).replace('600x900', 'blurred')) for im in blurred_images)
any(im.save(str(im.filename).replace('600x900', 'edges')) for im in contour_images)
any(im.save(str(im.filename).replace('600x900', 'sobel')) for im in sobel_images)

False